# 3. Simulation Results & HPC Scaling

In this notebook, we demonstrate:
- Running PDE simulations on different mesh sizes
- (Optionally) using MPI for parallel runs
- Recording runtime and memory usage

In [ ]:
import sys
import time
import dolfin
import numpy as np
import matplotlib.pyplot as plt
from models.thermal_model import ThermalModel

# Check if we are running in parallel (MPI)
mpi_comm = dolfin.MPI.comm_world
rank = mpi_comm.Get_rank()
size = mpi_comm.Get_size()

if rank == 0:
    print(f"Running on {size} MPI processes.")

## 3.1 Vary Mesh Size
We'll create a sequence of unit square meshes with increasing resolution: (20x20), (40x40), (80x80).

In [ ]:
mesh_sizes = [20, 40, 80]
runtimes = []

for msz in mesh_sizes:
    mesh = dolfin.UnitSquareMesh(msz, msz)
    model = ThermalModel(domain_mesh=mesh, rho=2700, cp=900, k=200, Q_electrical=0.0)
    model.set_initial_condition(700.0)
    model.set_boundary_conditions(700.0)

    start_t = time.time()
    # Simple run
    results = model.run_simulation(total_time=10, dt=1)
    end_t = time.time()
    elapsed = end_t - start_t

    # Collect times at rank=0
    runtimes.append(elapsed)
    if rank == 0:
        print(f"Mesh {msz}x{msz}, runtime: {elapsed:.2f} s")

## 3.2 Plot Performance

In [ ]:
if rank == 0:
    plt.figure()
    plt.plot(mesh_sizes, runtimes, marker='o')
    plt.title('Runtime vs. Mesh Size')
    plt.xlabel('Mesh Size')
    plt.ylabel('Runtime (s)')
    plt.grid(True)
    plt.show()